<a href="https://www.kaggle.com/code/golammostofas/making-bep-tokenizer-model?scriptVersionId=168958480" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install tokenizers


In [2]:
class CustomDecoder:
    def __init__(self):
        pass

    def decode(self, tokens):
        # Step 1: Replace "▁" with a space
        decoded = ' '.join(tokens).replace("▁", " ")
        
        # Step 2: ByteFallback (simplified example)
        # In a real scenario, you'd check if tokens are out of vocabulary and then decode them byte by byte.
        # This is a placeholder to illustrate where such logic would go.
        decoded = self.byte_fallback(decoded)
        
        # Step 3: Fuse (this example does not implement fusing logic as it's highly specific)
        # decoded = self.fuse(decoded)
        
        # Step 4: Strip leading spaces
        decoded = decoded.lstrip()
        
        return decoded
    
    def byte_fallback(self, text):
        # Placeholder for byte fallback logic
        # In a real scenario, you'd convert out-of-vocabulary tokens to their byte representation.
        return text

In [3]:
from tokenizers import Tokenizer, models, normalizers, pre_tokenizers, decoders, processors, trainers

# Initialize the tokenizer with a BPE model
tokenizer = Tokenizer(models.BPE(unk_token="[UNK]"))

# Normalization: Replacing spaces with a specified character
normalizer = normalizers.Sequence([
    normalizers.Replace(" ", "_"),
    normalizers.Prepend("_")
])
tokenizer.normalizer = normalizer

# Since you mentioned "pre_tokenizer": null, we won't set a pre-tokenizer
# Note: This might not be practical for most applications

# Post-Processor: Adding special tokens around sequences
tokenizer.post_processor = processors.TemplateProcessing(
    single="[CLS] $A [SEP]",
    pair="[CLS] $A [SEP] $B:1 [SEP]:1",
    special_tokens=[("[CLS]", 1), ("[SEP]", 2)]
)

# Assuming the "decoder": A custom decoding process isn't directly implemented here
tokenizer.decoder = decoders.Sequence([
    decoders.Replace("_", " "),  
    decoders.ByteFallback(),     # Fallback to byte encoding for unknown tokens
    decoders.Fuse(),             # Fuse tokens - specifics depend on implementation
    # Use Strip without 'start', or correctly according to its API.
    # This is speculative; you must replace it with the correct usage:
    decoders.Strip()  # Assuming Strip is to remove spaces or specific characters from tokens
])

# Initialize the trainer with special tokens
trainer = trainers.BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])

# Specify paths to your training files
files = ["/kaggle/input/romeo-and-juliet-tokenization/romeo-and-juliet_tokenization.txt"]

# Train the tokenizer
tokenizer.train(files, trainer)

# Save the tokenizer for later use
tokenizer.save("custom_tokenizer.json")


In [4]:
from tokenizers import Tokenizer

# Load the tokenizer
tokenizer = Tokenizer.from_file("/kaggle/working/custom_tokenizer.json")

# Encode some text
encoded = tokenizer.encode("Here is some text to encode_")

# Print the tokens and IDs
print(encoded.tokens)
print(encoded.ids)


['[CLS]', '_Here_is_', 'some_', 't', 'ex', 't_to_', 'en', 'co', 'de_', '[SEP]']
[1, 3855, 586, 76, 257, 1926, 100, 844, 807, 2]


In [5]:
# Assuming you've already loaded your tokenizer as shown previously
from tokenizers import Tokenizer

tokenizer = Tokenizer.from_file("/kaggle/working/custom_tokenizer.json")


# Decode the token IDs
decoded_text = tokenizer.decode(encoded.ids)

print(decoded_text)


 Here is some text to encode 
